Simple UI in Jupyter for labeling data (should've done this earlier)

In [ ]:
import pandas as pd
from IPython.display import display, clear_output
import ipywidgets as widgets

In [ ]:
# Load the test set
df = pd.read_parquet("../data/processed/test_2label_200_0.parquet")
df = df.copy()
df["label"] = ""

# Defining label options
label_options = [
    "action_required", "company_business", "purely_personal", "logistics",
    "employment", "newsletter", "spam", "empty", "fwd_chain"
]

def save_progress(df, path="../data/processed/test_labeled_200_0.parquet"):
    df.to_parquet(path, index=False)
    print(f"Progress saved to {path}")

start_idx = 0

for i in range(start_idx, len(df)):
    clear_output(wait=True)
    print(f"\n--- Email {i+1} of {len(df)} ---")
    
    subj = df.iloc[i]["subject"]
    body = df.iloc[i]["clean_body"]
    print(f"Subject:\n{subj}")
    print("\nBody:\n", body[:1000])

    dropdown = widgets.Dropdown(
        options=label_options,
        description='Label:',
        disabled=False,
    )

    button = widgets.Button(description="Submit")
    output = widgets.Output()

    def on_submit(b, idx=i):  # passing i to prevent errors
        df.iloc[idx, df.columns.get_loc("label")] = dropdown.value
        save_progress(df)
        with output:
            clear_output(wait=True)
            print(f"✅ Labeled email {idx + 1} as: {dropdown.value}")

    button.on_click(lambda b, idx=i: on_submit(b, idx))  # closure fix
    display(dropdown, button, output)

    input("Press Enter to continue to next email...")

    if (i + 1) % 10 == 0:
        save_progress(df)


In [ ]:
labeled = pd.read_parquet("../data/processed/test_labeled_200_0.parquet")
labeled.head(20)